In [23]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.preprocessing.text import text_to_word_sequence
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np
from bs4 import BeautifulSoup
import nltk
from sklearn import preprocessing
from keras.regularizers import l2
import re
from google.colab import drive
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D
from keras.layers import Dropout, BatchNormalization, LSTM
from keras.layers import Embedding

In [24]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# data = pd.read_csv('/content/drive/MyDrive/fake reviews dataset.csv',header=0,encoding_errors="ignore")
# df=data.drop(columns=['rating','category'],axis=1)

data = pd.read_csv('/content/drive/MyDrive/reviewContent+metadataBalanced.csv',header=0,encoding_errors="ignore")
df=data.drop(columns=['user_id','prod_id','rating','date'],axis=1)

In [26]:
data.head()

,user_id,prod_id,rating,label,date,text_
0,5044,0,1,0,11/16/2014,"Drinks were bad, the hot chocolate was watered..."
1,5045,0,1,0,9/8/2014,This was the worst experience I've ever had a ...
2,5046,0,3,0,10/6/2013,This is located on the site of the old Spruce ...
3,5047,0,5,0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...
4,5048,0,5,0,8/28/2014,I love Toast! The food choices are fantastic -...


In [27]:
df=df.dropna()
enc = preprocessing.LabelEncoder()
df['label']= enc.fit_transform(df['label'])

In [28]:
df.head()

,label,text_
0,0,"Drinks were bad, the hot chocolate was watered..."
1,0,This was the worst experience I've ever had a ...
2,0,This is located on the site of the old Spruce ...
3,0,I enjoyed coffee and breakfast twice at Toast ...
4,0,I love Toast! The food choices are fantastic -...


In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
re.compile('[/(){}\[\]\|@,;]')
re.compile('[^0-9a-z #+_]')

def cleaner_fun(text):

    try:
      text = BeautifulSoup(text, "lxml").text
    except:
      print(text)
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text

df['text_'] = df['text_'].apply(cleaner_fun)

<ipython-input-30-25c06feab4c9>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text


In [31]:
X = df.text_
y = df.label
X = X.map(lambda a: cleaner_fun(a))

In [32]:
def converter(text):
    text = re.sub(r'[^a-z\d ]', " ", text)
    text = re.sub(r'\d+', '_num_', text)
    return(text)

In [33]:
X = X.map(lambda a: converter(a))
train, test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.2,random_state=1)
tokenizer = Tokenizer(num_words=None,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split=' ',char_level=False)
tokenizer.fit_on_texts(X)
x_train = tokenizer.texts_to_sequences(train)
x_test = tokenizer.texts_to_sequences(test)
ind = tokenizer.word_index
X = tokenizer.texts_to_sequences(X)

In [34]:
v_len = len(ind)
avg = np.mean([len(seq) for seq in X])
sd = np.std([len(seq) for seq in X])
mx = int(avg + sd * 3)

In [35]:
x_train = pad_sequences(x_train, maxlen=mx)
x_test = pad_sequences(x_test, maxlen=mx)

In [36]:
dic = {}
f = open(os.path.join('/content/drive/MyDrive/', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    dic[word] = np.asarray(values[1:], dtype='float32')
f.close()

In [37]:
dimensions = 100
mat = np.zeros((len(ind) + 1, dimensions))
for word, i in ind.items():
    vec = dic.get(word)
    if vec is not None:
        mat[i] = vec

In [104]:
model = Sequential()
model.add(Embedding(v_len + 1, dimensions, weights=[mat], input_length=mx, trainable=True))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu',kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=4))
model.add(BatchNormalization())
model.add(LSTM(100,kernel_regularizer=l2(0.01),recurrent_regularizer=l2(0.01)))
model.add(Dense(1, activation='sigmoid',kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 412, 100)          8557000   
                                                                 
 dropout_3 (Dropout)         (None, 412, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 408, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 102, 64)           0         
 g1D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 102, 64)           256       
 chNormalization)                                                
                                                                 
 lstm_3 (LSTM)               (None, 100)              

In [105]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [106]:
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=5,batch_size=32,verbose=1)

Epoch 1/5
4024/4024 [==============================] - 126s 31ms/step - loss: 0.6986 - accuracy: 0.6753 - val_loss: 0.6126 - val_accuracy: 0.7176
Epoch 2/5
4024/4024 [==============================] - 59s 15ms/step - loss: 0.5861 - accuracy: 0.7357 - val_loss: 0.6576 - val_accuracy: 0.6689
Epoch 3/5
4024/4024 [==============================] - 55s 14ms/step - loss: 0.5449 - accuracy: 0.7642 - val_loss: 0.5901 - val_accuracy: 0.7259
Epoch 4/5
4024/4024 [==============================] - 56s 14ms/step - loss: 0.5112 - accuracy: 0.7858 - val_loss: 0.6540 - val_accuracy: 0.6837
Epoch 5/5
4024/4024 [==============================] - 55s 14ms/step - loss: 0.4833 - accuracy: 0.8009 - val_loss: 0.6079 - val_accuracy: 0.7249


In [107]:
model.evaluate(x_test,y_test)

1006/1006 [==============================] - 4s 4ms/step - loss: 0.6079 - accuracy: 0.7249


[0.6079093217849731, 0.7248889207839966]

In [108]:
y_pred = list(model.predict(x_test))

1006/1006 [==============================] - 4s 4ms/step


In [109]:
y_pred_final=[]
for i in range(len(y_pred)):
  if y_pred[i][0]<0.5:
    y_pred_final.append(0)
  else:
    y_pred_final.append(1)

In [110]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_final))

              precision    recall  f1-score   support

           0       0.69      0.81      0.75     16093
           1       0.77      0.64      0.70     16094

    accuracy                           0.72     32187
   macro avg       0.73      0.72      0.72     32187
weighted avg       0.73      0.72      0.72     32187

